In [2]:
import os
import torch
import mlflow
import mlflow_env
from gpt import GPTLanguageModel
from autoencoder import Autoencoder
from gpt_params import tokenizer
from torch.nn import functional as F
from gpt_params import transformer_experiment
from autoencoder_params import autoencoder_experiment, device


In [3]:
transformer_run_id = "1631cdf63904427fb5833afa9372b625"
autoencoder_run_id = "fead6458c7d24cd9a86a3466dbf09dd4"

gpt = GPTLanguageModel.load_from_mlflow(transformer_experiment, transformer_run_id, device)
autoencoder = Autoencoder.load_from_mlflow(autoencoder_experiment, autoencoder_run_id, device)

gpt.eval()
autoencoder.eval()

/home/test/nlp/NLP-monosemanticity/gpt.py:151: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(local_model_path, map_location=device)



GPT loaded from MLflow:

Metrics:
  cross_entropy_loss_train: 4.2946
  cross_entropy_loss_eval: 4.7007
  interval_time: 5.6415

Parameters:
  Dataset: wikitext-wikitext-103-v1
  subsets_max_size: 20
  num_training_subsets: 50
  epochs: 1
  learning_rate: 0.001
  batch_size: 768
  optimizer: AdamW
  context_length: 40
  embedding_dim: 128
  num_of_attention_heads: 8
  num_of_blocks: 1
  vocab_size: 50257
  dropout: 0



Autoencoder loaded from MLflow:

Metrics:
  loss_train: 0.0203
  loss_eval: 0.0196
  recon_loss_eval: 0.0126
  norm_loss_eval: 0.007
  acts_eval: 439.5797
  acts_eval_percentage: 42.927703857421875

Parameters:
  lasso_lambda: 0.0001
  learning_rate: 0.0001
  num_epochs: 1
  batch_size: 64
  transformer_model_run_id: 1631cdf63904427fb5833afa9372b625
  sparse_dimension_factor: 8
  num_training_subsets: 5
  subsets_max_size: 20



/home/test/nlp/NLP-monosemanticity/autoencoder.py:57: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(local_model_path, map_location=device)


Autoencoder(
  (encoder): Linear(in_features=128, out_features=1024, bias=True)
  (decoder): Linear(in_features=1024, out_features=128, bias=True)
  (relu): ReLU()
)

Forcing a neuron

In [19]:
def biased_generate(idx, max_new_tokens, neuron_idx, activation):
    if neuron_idx < 0 or neuron_idx >= autoencoder.dim_rala:
        raise ValueError(f"Invalid neuron index: {neuron_idx}")

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -gpt.context_length :]
        x = gpt.embed(idx_cond)
        encoded, _ = autoencoder(x)
        encoded[:, :, neuron_idx] += activation
        x = autoencoder.decode(encoded)
        logits = gpt.unembed(x)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx

In [24]:
idx = (
    torch.tensor(tokenizer.encode("We are going to"), dtype=torch.long)
    .unsqueeze(0)
    .to(device)
)
out = biased_generate(idx, 100, 729, 0.86)

print(tokenizer.decode(out.squeeze().tolist()))

We are going to happen at Quainton . Although the judge 's actions are strained in the scene except this . 


 = = Event = = = 


 On the episode of Earth bound for Chattanooga , Montana , the network 's puzzle , " Echoes " , ( 2000 ) . The conclusion of the series ends at the start of his race , taken on a screen so that he did not only participate in the top five spinly in the 18 – 28 episode , roughly 5 @.@ 66
